In [31]:
import os
import cv2
import threading
from mtcnn import MTCNN
from deepface import DeepFace

In [ ]:
test_img = cv2.imread('known_faces/Nathan.jpg')

test_result = DeepFace.extract_faces(img_path='known_faces/Nathan.jpg')

print(test_result[0]['facial_area'])

In [3]:
for face in test_result:
    facial_area = face['facial_area']
    x = facial_area["x"]
    y = facial_area["y"]
    w = facial_area["w"]
    h = facial_area["h"]

    cur_face = test_img[y:y+h, x:x+w]
    cv2.imshow('current face', cur_face)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Draw the rectangle on the image
    cv2.rectangle(test_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv2.imshow('Detected', test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# using MTCNN through deepface
it is slow bud

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success:
        break

    facesCurFrame = DeepFace.extract_faces(img.copy(), detector_backend="mtcnn", enforce_detection=False)
    
    for faceLoc in facesCurFrame:
        facial_area = faceLoc.get("facial_area", {})
        x = facial_area.get("x", 0)
        y = facial_area.get("y", 0)
        w = facial_area.get("w", 0)
        h = facial_area.get("h", 0)

        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('Webcam', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Using MTCNN directly
wayyyy faster

In [39]:
detector = MTCNN(device="CPU:0")

result = detector.detect_faces(test_img)

print(result[0]['box'][0])

577


In [43]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success:
        break

    facesCurFrame = detector.detect_faces(img)
    
    for faceLoc in facesCurFrame:
        cv2.rectangle(img, (faceLoc['box'][0], faceLoc['box'][1]), (faceLoc['box'][0] + faceLoc['box'][2], faceLoc['box'][1] + faceLoc['box'][3]), (0, 255, 0), 2)

    cv2.imshow('Webcam', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# MTCNN along with recognizer
in this one i will be using facenet

In [52]:
import os
from PIL import Image
import numpy as np
from matplotlib import pyplot
from keras.models import load_model
from keras_facenet import FaceNet

import pickle
import cv2

In [53]:
# re declare the detectors
detector = MTCNN(device="CPU:0")

result = detector.detect_faces(test_img)

print(result[0]['box'][0])

577


In [57]:
# what if it detects a no-face image

no_face_image = cv2.imread('known_faces/unknown.jpg')

no_face_result = detector.detect_faces(no_face_image)

print(len(no_face_result))

0


In [59]:
# what if it detects a face image

no_face_image = cv2.imread('known_faces/Bapallua.jpg')

no_face_result = detector.detect_faces(no_face_image)

print(len(no_face_result))
print(no_face_result)

1
[{'box': [451, 351, 177, 221], 'confidence': np.float64(0.9987026453018188), 'keypoints': {'nose': [np.int64(553), np.int64(490)], 'mouth_right': [np.int64(589), np.int64(526)], 'right_eye': [np.int64(588), np.int64(432)], 'left_eye': [np.int64(506), np.int64(443)], 'mouth_left': [np.int64(527), np.int64(534)]}}]


In [64]:
MyFaceNet = FaceNet()

In [68]:
folder = 'known_faces/'
database = {}

for filename in os.listdir(folder):
    path = folder + filename
    curimg = cv2.imread(folder+filename)

    det_res = detector.detect_faces(curimg)

    if len(det_res) > 0:
        x, y, w, h = det_res[0]['box'][0], det_res[0]['box'][1], det_res[0]['box'][2],det_res[0]['box'][3]
    else:
        continue

    # cv2.rectangle(curimg, (x, y), (x+w, y+h), (0,255,0), 2)
    # cv2.imshow(filename, curimg)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows

    img_input = cv2.cvtColor(curimg, cv2.COLOR_BGR2RGB)
    img_input = Image.fromarray(img_input)
    img_array = np.asarray(img_input)

    face = img_array[y:y+h, x:x+w]
    face = Image.fromarray(face)
    face = face.resize((160,160))
    face = np.asarray(face)

    face = np.expand_dims(face, axis = 0)
    signature = MyFaceNet.embeddings(face)

    database[os.path.splitext(filename)[0]]=signature

database


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


{'Bapallua': array([[ 0.02359171, -0.00668691, -0.01756126,  0.04090104,  0.01974503,
          0.00222332, -0.03010758, -0.02478369,  0.08800139,  0.02033452,
          0.03611914,  0.08760065,  0.01691404,  0.03410093, -0.07625847,
         -0.00713598,  0.02753266,  0.03213055,  0.01174737,  0.01989158,
         -0.00063227,  0.0602023 ,  0.08331351, -0.05148023, -0.04122752,
          0.0066965 , -0.00500664, -0.03456511, -0.00995303, -0.02919886,
          0.04922036,  0.06296912,  0.04044762, -0.00853496,  0.00990689,
         -0.02640278, -0.0372687 , -0.08084124, -0.01671006,  0.01433043,
          0.02390222, -0.09141319, -0.05028492, -0.00178137, -0.06978234,
          0.01577557, -0.04750962,  0.04669497, -0.05438142, -0.03245326,
         -0.04413438, -0.10284945,  0.02092307,  0.01359168, -0.01837446,
         -0.028386  ,  0.00923796, -0.00586575,  0.03605784, -0.012345  ,
         -0.02032346,  0.06589191, -0.00097742,  0.11720493, -0.00643191,
         -0.00441547, -0.0

In [69]:
myfile = open("data.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()

In [ ]:
# cap = cv2.VideoCapture(0)

# while True:
#     success, img = cap.read()
#     if not success:
#         break

#     facesCurFrame = detector.detect_faces(img)
    
#     for faceLoc in facesCurFrame:
#         cv2.rectangle(img, (faceLoc['box'][0], faceLoc['box'][1]), (faceLoc['box'][0] + faceLoc['box'][2], faceLoc['box'][1] + faceLoc['box'][3]), (0, 255, 0), 2)

#     cv2.imshow('Webcam', img)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()